In [1]:
# check if you are in the right country to use Gemini api keys
!curl ipinfo.io

{
  "ip": "27.131.212.33",
  "hostname": "ns0.wishnet.in",
  "city": "Kolkata",
  "region": "West Bengal",
  "country": "IN",
  "loc": "22.5626,88.3630",
  "org": "AS45775 WISH NET PRIVATE LIMITED",
  "postal": "700001",
  "timezone": "Asia/Kolkata",
  "readme": "https://ipinfo.io/missingauth"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   296  100   296    0     0    855      0 --:--:-- --:--:-- --:--:--   875
100   296  100   296    0     0    852      0 --:--:-- --:--:-- --:--:--   873


In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time

def extract_data(driver):
    # Wait for the listings to load (adjust timeout as needed)
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'Box__Div-sc-dws99b-0 iOfhmk')))
    except TimeoutException:
        print("Timed out waiting for listings to load.")
        return

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')

    company_elements = soup.find_all('div', class_='Box__Div-sc-dws99b-0 iOfhmk')

    for company in company_elements:
        # --- Company Name ---
        company_name_element = company.find('h3', class_='MuiTypography-root MuiTypography-h3 MuiTypography-displayBlock')
        company_names.append(company_name_element.text.strip() if company_name_element else 'N/A')

        # --- Location ---
        location_element = company.find('p', class_='MuiTypography-root MuiTypography-body2 MuiTypography-colorTextSecondary')
        locations.append(location_element.text.strip() if location_element else 'N/A')

        # --- Phone Number ---
        phone_number_element = company.find('a', class_='MuiButtonBase-root MuiButton-root MuiButton-text ButtonPhone MuiButton-textPrimary MuiButton-fullWidth')
        phone_numbers.append(phone_number_element.text.strip() if phone_number_element else 'N/A')

        # --- Website URL ---
        website_url_element = company.find('a', class_='MuiButtonBase-root MuiButton-root MuiButton-text ButtonWebsite MuiButton-textSecondary MuiButton-fullWidth')
        website_urls.append(website_url_element['href'] if website_url_element else 'N/A')

        # --- Description ---
        description_element = company.find('p', class_='MuiTypography-root MuiTypography-subtitle2 MuiTypography-paragraph')
        descriptions.append(description_element.text.strip() if description_element else 'N/A')

        # --- Rating ---
        rating_element = company.find('div', class_='Box__Div-sc-dws99b-0 iiSpqr')
        ratings.append(rating_element.text.strip() if rating_element else 'N/A')

        # --- Reviews Count ---
        reviews_count_element = company.find('p', class_='MuiTypography-root MuiTypography-body2')
        reviews_count.append(reviews_count_element.text.strip() if reviews_count_element else 'N/A')

        # --- Opening Hours ---
        opening_hours_element = company.find('span', class_='jss391')
        opening_hours.append(opening_hours_element.text.strip() if opening_hours_element else 'N/A')

        # --- Licences ---
        licence_element = company.find('p', class_='MuiTypography-root MuiTypography-body2 MuiTypography-colorSecondary')
        licences.append(licence_element.text.split(":")[-1].strip() if licence_element else 'N/A')

        # --- Features ---
        feature_elements = company.find_all('div', class_='Box__Div-sc-dws99b-0 bKFqNV jss416')
        features_list = [feature.text.strip() for feature in feature_elements]
        features.append(features_list)

# Initialize data lists outside the function to make them global
company_names = []
locations = []
phone_numbers = []
website_urls = []
descriptions = []
ratings = []
reviews_count = []
opening_hours = []
licences = []
features = []

# Configure Selenium
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Optional: Run in headless mode 
driver = webdriver.Chrome(options=options)

# Starting URL
base_url = 'https://www.yellowpages.com.au/search/listings?clue=Electricians&locationClue=Australia&pageNumber='
start_page = 1
driver.get(base_url + str(start_page))

# Extract data from the first page
extract_data(driver)

# Get total pages from the pagination element
pagination_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, 'Box__Div-sc-dws99b-0 zzTlQ')))
total_pages = int(pagination_element.find('p').text.split()[-1])

# Loop through remaining pages
for page_number in range(start_page + 1, total_pages + 1):
    print(f"Scraping page {page_number}...")
    driver.get(base_url + str(page_number))

    extract_data(driver)

    time.sleep(2)  # Be respectful of the website's load

driver.quit()

# Create DataFrame
df = pd.DataFrame({
    'Company Name': company_names,
    'Location': locations,
    'Phone Number': phone_numbers,
    'Website URL': website_urls,
    'Description': descriptions,
    'Rating': ratings,
    'Reviews Count': reviews_count,
    'Opening Hours': opening_hours,
    'Licence': licences,
    'Features': features
})

print(df)
df.to_csv("yellow_pages_electricians.csv", index=False)

Timed out waiting for listings to load.


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF67A7EEEA2+31554]
	(No symbol) [0x00007FF67A767ED9]
	(No symbol) [0x00007FF67A62872A]
	(No symbol) [0x00007FF67A678434]
	(No symbol) [0x00007FF67A67853C]
	(No symbol) [0x00007FF67A6BF6A7]
	(No symbol) [0x00007FF67A69D06F]
	(No symbol) [0x00007FF67A6BC977]
	(No symbol) [0x00007FF67A69CDD3]
	(No symbol) [0x00007FF67A66A33B]
	(No symbol) [0x00007FF67A66AED1]
	GetHandleVerifier [0x00007FF67AAF8B1D+3217341]
	GetHandleVerifier [0x00007FF67AB45AE3+3532675]
	GetHandleVerifier [0x00007FF67AB3B0E0+3489152]
	GetHandleVerifier [0x00007FF67A89E776+750614]
	(No symbol) [0x00007FF67A77375F]
	(No symbol) [0x00007FF67A76EB14]
	(No symbol) [0x00007FF67A76ECA2]
	(No symbol) [0x00007FF67A75E16F]
	BaseThreadInitThunk [0x00007FF99865257D+29]
	RtlUserThreadStart [0x00007FF9994AAF28+40]
